In [ ]:
%%bash
# install dependencies
apt install -y autoconf2.13 autotools-dev bison flex g++ gettext git imagemagick libblas-dev libbz2-dev libcairo2-dev libfftw3-dev libfreetype6-dev libgdal-dev libgeos-dev libglu1-mesa-dev libjpeg-dev liblapack-dev liblas-c-dev libncurses5-dev libnetcdf-dev libpng-dev libpq-dev libproj-dev libreadline-dev libsqlite3-dev libtiff-dev libxmu-dev libzstd-dev make netcdf-bin p7zip proj-bin sqlite3 unixodbc-dev xvfb zlib1g-dev

pip install Pillow
pip install ply
pip install PyVirtualDisplay

git clone --depth 1 --branch Jupyter-Timeseries https://github.com/chaedri/grass.git grass-src

# enter the directory with source code
cd grass-src

# compile
./configure \
    --enable-largefile=yes \
    --with-nls \
    --with-cxx \
    --with-readline \
    --with-bzlib \
    --with-pthread \
    --with-proj-share=/usr/share/proj \
    --with-geos=/usr/bin/geos-config \
    --with-cairo \
    --with-opengl-libs=/usr/include/GL \
    --with-freetype=yes --with-freetype-includes="/usr/include/freetype2/" \
    --with-sqlite=yes
make -j2
make -j2 install

# leave the directory with source code
cd ..

# download sample data
mkdir -p grassdata
wget http://fatra.cnr.ncsu.edu/foss4g2021/dsm.tif
wget http://fatra.cnr.ncsu.edu/foss4g2021/ground.tif
wget http://fatra.cnr.ncsu.edu/foss4g2021/ortho.tif
wget http://fatra.cnr.ncsu.edu/foss4g2021/ortho.tif.aux.xml
wget http://fatra.cnr.ncsu.edu/foss4g2021/roads_backup.geojson -O roads.geojson

# Part 1: GRASS GIS intro

In this first part we will demonstrate starting GRASS GIS, creating new project and basic data visualization.
We will first use the GUI and then we will replicate it in the notebook environment running on mybinder.com.

## GUI intro

## GRASS GIS in a notebook

### Notebook environment

By default all cells are running Python:

In [ ]:
import sys

v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

However, we will also show how to use GRASS from Bash shell, for that we will use IPython magic:

In [ ]:
%%bash
pwd

### Start GRASS GIS

Create new empty location (project) that uses projection [UTM zone 17 N](https://epsg.io/6346). Here, we place it directory called _grassdata_ under the home directory which already exists in this Binder environment (create it if you are running the notebook locally and change the path here accordingly).

In [ ]:
%%bash
grass -c EPSG:6346 -e ~/grassdata/dix_park/ 

Start GRASS GIS session in the newly created location. Load Python libraries.

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata/dix_park/PERMANENT")

### Import data

Import prepared DSM, DTM and ortho maps. If you run this notebook locally (not in Binder), download the data [here](http://fatra.cnr.ncsu.edu/foss4g2021/). In this case no reprojection is done since the data CRS matches the location's CRS.

We also set the computational region based on the imported raster (more on that later).

In [ ]:
gs.run_command("r.import", input="dsm.tif", output="dsm", resample="bilinear")
gs.run_command("r.import", input="ground.tif", output="ground")
gs.run_command("r.import", input="ortho.tif", output="ortho")

# set the region to match ortho raster
gs.run_command("g.region", raster="ortho")

`roads.geojson` We extract roads for our study area using Download OSM data using Overpass Turbo.
The data was <https://overpass-turbo.eu/s/1aGu> The query should run upon running the cell below, so
it should be enough to *press Export - download as GeoJSON* and save it as `roads.geojson`.
This will download the file on your local machine, so if you are running this in Jupyter Lab on Binder, upload it there with *Upload files* in the top left corner.

Now we import the downloaded file. This time the data will be automatically reprojected because the CRS doesn't match.

In [ ]:
%%bash
v.import input=roads.geojson output=roads --quiet

Let's look at the available data in our location:

In [ ]:
%%bash
g.list type=raster,vector -m -t

### Data visualization

We will call d.rast/d.vect modules using the new GRASS Jupyter API:

In [ ]:
# Start a GrassRenderer
img = gj.GrassRenderer()
# Add a raster and vector to the map
img.d_rast(map="ground")
img.d_vect(map="roads")
img.d_legend(raster="ground", flags="b")
# Display map
img.show()

In [ ]:
img = gj.GrassRenderer()
img.d_rast(map="ortho")
img.d_vect(map="roads", width="2", color="yellow", where="name = 'Umstead Drive'")
img.show()

Here is how we can visualize data with folium:

In [ ]:
fig = gj.InteractiveMap(width=600)
# Add vector and layer control to map
fig.add_vector("roads")
fig.show()

We can also visualize data in 3D. Here we drape the ortho over the DSM.

In [ ]:
img = gj.Grass3dRenderer()
img.render(
    elevation_map="dsm",
    color_map="ortho",
    position=(0.5, 1),
    height=3000,
    perspective=12,
)
img.show()

In [ ]:
img = gj.Grass3dRenderer()
img.render(
    elevation_map="dsm",
    resolution_fine=1,
    color_map="ortho",
    light_position=(1, 0, 0.5),
    position=(0.75, 0.35),
    height=1500,
    perspective=10,
)
img.show()

## GRASS GIS modules

GRASS functionality is available through modules (tools, functions). There are over 500 different modules in the core distribution and over 300 addon modules that can be used to prepare and analyze data.

Modules respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some modules have multiple dots in their names. For example, modules staring with v.net.* deal with vector network analysis and r.in.* modules import raster data into GRASS GIS spatial database.


### Finding and running a module

![Find modules](img/search_module.gif)

To find a module for your analysis, type the term into the **search box within the Modules tab** in the Layer Manager or just browse the module tree.
Most modules are also available from the **main menu**. For example, to find information about a raster map, use: *Raster → Reports and statistics → Basic raster metadata*.

If you already know the name of the module, you can just use it in the command line. The GUI offers a **Console tab with command line** specifically built for running GRASS GIS modules. If you type the module name there, you will get suggestions for automatic completion of the name. After pressing Enter, you will get the GUI dialog for the module.

Last but not least, there is a module for finding modules:

In [ ]:
%%bash
g.search.modules keyword=zonal

In [ ]:
%%bash
r.univar --help

### GUI  vs command line vs Python

GRASS modules can be executed either through the GUI, command line or Python interfaces. The GUI offers a user-friendly approach to execute modules where the user can navigate to data layers that they would like to analyze and modify processing options with simple check boxes.
The GUI also offers an easily accessible manual on how to execute a model. The command line interface allows users to execute a module using command prompts specific to that module. This is handy when you are running similar analyses with minor modification or are familiar with the module commands for quick efficient processing.

In this example we will show how the same module can be executed with the GUI, in shell and in Python:

![v.extract](img/v.extract.png) &nbsp;&nbsp;&nbsp;&nbsp;
![v.extract](img/v.extract.gif)

In [ ]:
%%bash
v.extract input=roads where="name = 'Umstead Drive'" output=umstead_drive_segments

In [ ]:
gs.run_command(
    "v.extract",
    input="roads",
    where="name = 'Umstead Drive'",
    output="umstead_drive_segments",
)

### Computational region

Computational region is an important raster concept in GRASS GIS.
Before we use a module to compute a new raster map, we must properly set the computational region. All raster computations will be performed in the specified extent and with the given resolution.
Among other things, this allows us to easily subset larger extent data for quicker testing of analysis, or to run an analysis of specific regions given by e.g. administrative units.

A few points to keep in mind:

 * computational region is defined by region extent and raster resolution
 * applies to all raster operations and vector to raster operations
 * persists between GRASS sessions, can be different for different mapsets
 * advantages: keeps your results consistent, avoids clipping, for computationally demanding tasks set region to smaller extent, check that your result is good and then set the computational region to the entire study area and rerun analysis
 * run `g.region -p` or in menu *Settings - Region - Display region* to see current region settings
 
 ![Computational region](img/region.gif)

In [ ]:
%%bash
g.region -p

The most common way to set region is **based on a raster map** - both extent and resolution. If the raster is added to Layer Manager, right click on it and select *Set computational region from selected map*. Or run the g.region module (we include -p flag to always see the resulting region):

In [ ]:
gs.run_command("g.region", raster="dsm")

Computational region can be set also **using a vector map**. In that case, only extent is set (as vector maps do not have any resolution - at least not in the way raster maps do). In the GUI, this can be done in the same way as for the raster map. In the command line, it looks like this:


In [ ]:
%%bash
g.region -p vector=roads | tail -n 10

However now the resolution was adjusted based on the extent of the vector map, it is no longer a nice rounded number. If that's not desired, we can set it explicitly using -a flag and parameter res. Now the resolution is aligned to even multiples of 2 (the units are the units of the current location, in our case meters):


In [ ]:
print(gs.read_command("g.region", vector="roads", res=2, flags="ap"))

Often we need to set the computational extent based on a vector map, but take the resolution and alignment from a raster map:

In [ ]:
gs.parse_command("g.region", vector="roads", align="dsm", flags="g")

## Python API

There are two Python APIs for accessing a module's functionality - [GRASS GIS Python Scripting Library](https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html) (_grass.script_) and [PyGRASS](https://grass.osgeo.org/grass78/manuals/libpython/pygrass_index.html) (_grass.pygrass_).
PyGRASS is advantageous for more advanced workflows. Here we will be using Python Scripting Library (`import grass.script as gs`)
as it is simple and straightforward to use.
We are also using an API for Jupyter (`import grass.jupyter as gj`) which makes our workflows in notebooks more convenient.

GRASS GIS Python Scripting Library provides functions to call GRASS modules within scripts as subprocesses. The most often used functions include:

 * [script.core.run_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.run_command): used with modules which output raster/vector data where text output is not expected
 * [script.core.read_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.read_command): used when we are interested in text output
 * [script.core.parse_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.parse_command): used with modules producing text output as key=value pair
 * [script.core.write_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.write_command): for modules expecting text input from either standard input or file


It also provides several wrapper functions for common modules to simplify basic tasks. The list of convenient wrapper functions with examples includes:

 * Raster metadata using [script.raster.raster_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.raster_info): `gs.raster_info('dsm')`
 * Vector metadata using [script.vector.vector_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.vector.vector_info): `gs.vector_info('roads')`
 * List raster data in current location using [script.core.list_grouped()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.list_grouped): `gs.list_grouped(type=['raster'])`
 * Get current computational region using [script.core.region()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region): `gs.region()`

**Note for GUI:** The simplest way, especially on Windows, to execute the Python code which uses GRASS GIS packages is to use the _Simple Python Editor_ integrated in GRASS GIS (accessible from the toolbar or the _Python_ tab in the _Layer Manager_). Another option is to write the Python code in your favorite plain text editor like _Notepad++_ (note that Python editors are plain text editors). Then run the script in GRASS GIS using the main menu *File -> Launch script*.

![Run Python in GUI](img/python.gif)

# Part 2: Viewshed case study

In the second part we will demonstrate the use of GRASS for a small viewshed case study.
The goal is to compute and analyze the area a driver would see from a road.

Topics covered:
 * viewshed computation ([r.viewshed](https://grass.osgeo.org/grass-stable/manuals/r.viewshed.html))
 * Python scripting
 * manipulating vector data ([v.build.polylines](https://grass.osgeo.org/grass-stable/manuals/v.build.polylines.html), [v.to.points](https://grass.osgeo.org/grass-stable/manuals/v.to.points.html))
 * vector attributes ([v.db.select](https://grass.osgeo.org/grass-stable/manuals/v.db.select.html))
 * simple parallelization ([multiprocessing.Pool](https://docs.python.org/3/library/multiprocessing.html))
 * region handling ([grass.script.region_env](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region_env))
 * raster algebra ([r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html))
 * reprojecting ([r.proj](https://grass.osgeo.org/grass-stable/manuals/r.proj.html))
 * resampling ([r.resample.interp](https://grass.osgeo.org/grass-stable/manuals/r.resample.interp.html))
 * raster mask ([r.mask](https://grass.osgeo.org/grass-stable/manuals/r.mask.html))
 * raster as numpy array ([grass.script.array](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#module-script.array))

In [ ]:
%%bash
grass -e -c ~/grassdata/dix_park/viewsheds

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys

# Import package for progress bars, see binder/requirements.txt.
from tqdm import tqdm

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata/dix_park/viewsheds")

## Compute viewshed and visible area

Set computational region based on DSM:

In [ ]:
gs.run_command("g.region", raster="dsm")

Compute viewshed, flag 'b' is for binary (0 and 1) output:

In [ ]:
gs.run_command(
    "r.viewshed",
    input="dsm",
    output="viewshed",
    flags="b",
    coordinates=(711260, 3960860),
)

Compute basic univariate statistics, flag 'g' is for parsable output:

In [ ]:
univar = gs.parse_command("r.univar", map="viewshed", flags="g")

Get current region settings to get cell size:

In [ ]:
region = gs.region()
cell_size = region["nsres"] * region["ewres"]
percentage = 100 * float(univar["sum"]) / float(univar["n"])
area = cell_size * float(univar["sum"])
print(f"Percentage of visible area is {percentage:.2f}%, which is {area} ha")

In [ ]:
img = gj.GrassRenderer()
img.d_rast(map="ortho")
# select only cells with value 1 to visualize
img.d_rast(map="viewshed", values=1)
img.show()

### Data preparation
We will first derive viewpoints along the road *Umstead Drive* (vector `umstead_drive_segments`) that we extracted in the first part of the workshop.

1. Because the road consists of several segments, we will merge them into one.
2. Create new vector of points along the line with distance 50 m.

In [ ]:
gs.run_command(
    "v.build.polylines",
    input="umstead_drive_segments",
    output="umstead_drive",
    cats="first",
)
gs.run_command(
    "v.to.points", input="umstead_drive", type="line", output="viewpoints", dmax=50
)

In [ ]:
gs.run_command("g.region", raster="dsm")

We want to compute the visibility using DSM, however some points may fall on top of a tree, so we need to filter those out.

3. Compute height above ground (DSM - DTM).
4. Extract height above ground for the viewpoint locations.

In [ ]:
gs.mapcalc("diff = dsm - ground")
gs.run_command("v.what.rast", map="viewpoints", layer=2, raster="diff", column="height")
gs.run_command("r.colors", map="diff", color="differences")

See the newly computed attribute data. This example shows how the attribute data can be loaded into pandas:

In [ ]:
import json
import pandas as pd

pd.DataFrame(
    json.loads(
        gs.read_command(
            "v.db.select",
            map="viewpoints",
            columns="cat,height",
            layer=2,
            format="json",
        )
    )["records"]
)

Visualize the viewpoints with the height-above-ground raster. You can filter the points based on the height:

In [ ]:
img = gj.GrassRenderer()
img.d_rast(map="diff")
img.d_vect(map="umstead_drive")
img.d_vect(map="viewpoints", layer=2, where="height < 2", size=10, icon="basic/pin")
img.d_legend(raster="diff")
img.show()

### Viewshed computation
To get the cumulative viewshed, we will compute viewsheds from all the viewpoints we generated earlier.
First, we get the list coordinates of the viewpoints that are likely lying on the ground:

In [ ]:
viewpoints = (
    gs.read_command(
        "v.out.ascii",
        input="viewpoints",
        separator="comma",
        layer=2,
        where="height < 2",
    )
    .strip()
    .splitlines()
)
viewpoints = [p.split(",") for p in viewpoints]
viewpoints

We will now compute the viewshed from each viewpoint in a loop. We set max distance of 300 m. Each viewshed will be named `viewshed_cat`.

In [ ]:
%%time
maps = []
for x, y, cat in tqdm(viewpoints):
    name = f"viewshed_{cat}"
    gs.run_command(
        "r.viewshed", input="dsm", output=name, coordinates=(x, y), max_distance=300
    )
    maps.append(name)

Since these are independent runs, we can easily parallelize the r.viewshed calls using Python multiprocessing.
We define a function that computes the viewshed and returns the name of the output or None in case of error:

List of map names is ordered based on the points on the road (this will be important later on):

In [ ]:
print(maps)

One trick to speedup viewshed computation is to limit the computation only
to the actual area given by the maxdistance option. To do that we will locally modify the computational region
and pass the environment to the module directly. The current computational region won't be affected.

In [ ]:
%%time
for point in tqdm(viewpoints):
    x, y, cat = point
    x, y = float(x), float(y)
    max_distance = 300
    # copy current environment
    env = os.environ.copy()
    # set GRASS_REGION variable using region_env function
    env["GRASS_REGION"] = gs.region_env(
        align="dsm",
        e=x + max_distance,
        w=x - max_distance,
        n=y + max_distance,
        s=y - max_distance,
    )
    name = f"viewshed_{cat}"
    gs.run_command(
        "r.viewshed",
        input="dsm",
        output=name,
        coordinates=(x, y),
        max_distance=max_distance,
        env=env,
    )

See the difference in number of cells between one of the viewsheds and the DSM:

In [ ]:
print(f"Viewshed num cells: {gs.raster_info(maps[0])['cells']}")
print(f"DSM num cells: {gs.raster_info('dsm')['cells']}")

### Cumulative viewshed
Finally, we can compute the cumulative viewshed, which aggregates viewsheds from multiple viewpoints. In this way you can e.g., identify the most frequently visible areas from the road.

First, let's check we have the viewshed rasters ready:

In [ ]:
gs.list_strings(type="raster", pattern="viewshed_*", mapset=".")

Since by default viewshed rasters have no data in areas that are not visible, we will use r.series method *count*:

In [ ]:
# cumulative viewshed
gs.run_command("r.series", input=maps, output="cumulative_viewshed", method="count")
# set color of cumulative viewshed from grey to yellow
color_rules = """\
0% 70:70:70
100% yellow
"""
gs.write_command("r.colors", map="cumulative_viewshed", rules="-", stdin=color_rules)

Let's visualize the results in an interactive map:

In [ ]:
fig = gj.InteractiveMap(width=600)
# Add vector and layer control to map
fig.add_raster("cumulative_viewshed", opacity=0.6)
fig.add_vector("umstead_drive")
fig.add_layer_control(position="bottomright")
fig.show()

And create a 3D rendering with draped cumulative viewshed over the DSM:

In [ ]:
img = gj.Grass3dRenderer()
img.render(
    elevation_map="dsm",
    resolution_fine=1,
    color_map="cumulative_viewshed",
    vline="umstead_drive",
    vline_width=3,
    vline_color="white",
    position=[0.5, 0.8],
    height=2500,
    perspective=15,
)
img.overlay.d_legend(
    raster="cumulative_viewshed", at=(60, 97, 87, 92), color="white", flags="f"
)
img.show()

## Timeseries

In [ ]:
gs.run_command(
    "t.create",
    output="viewshed_walk",
    type="strds",
    temporaltype="absolute",
    title="Average temperature",
    description="Monthly temperature average in NC [deg C]",
)
gs.run_command(
    "t.register",
    input="viewshed_walk",
    type="raster",
    start="2000-01-01 14:00",
    increment="5 minutes",
    maps=maps,
    flags="i",
)

In [ ]:
color_rules = """\
0% red
100% yellow
"""
gs.write_command("t.rast.colors", input="viewshed_walk", rules="-", stdin=color_rules)

In [ ]:
img = gj.TimeSeries("viewshed_walk")
img.baselayer.d_rast(map="ortho")
img.time_slider()

In [ ]:
img.animate(duration=500, label=True, text_size=16, text_color="gray")